In [1]:
from gmath import *
from process import *

[Taichi] version 1.6.0, llvm 15.0.1, commit f1c6fbbd, win, python 3.9.13
[Taichi] Starting on arch=cuda


In [2]:
pixels = load_data()
c2w = load_mat()
rays = ti.Vector.field(3,ti.f32, shape=(len(pixels), pixels[0].shape[0], pixels[0].shape[1]))
origins = ti.Vector.field(3,ti.f32, shape=(len(pixels)))
print(len(pixels),c2w.shape,rays.shape,origins.shape)

Loading Lego/rgb/


 25%|██▌       | 100/400 [00:04<00:14, 20.97it/s]


Loading Lego/pose/


 25%|██▌       | 100/400 [00:00<00:00, 4346.65it/s]


100 (100,) (100, 800, 800) (100,)


In [3]:
calRays(0,rays,c2w,origins)

In [6]:
rays[0,400,400]

[ 0.01209058 -0.95358515 -0.3008804 ]

In [4]:
gui = ti.GUI('window', (800,800), fast_gui=False)
while gui.running:
    gui.set_image(pixels[0])
    gui.show()